In [17]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_prominent_mountains_of_the_Alps_above_3000_m")

In [19]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [20]:
tables = soup.findAll("table")

In [21]:
## googlato "python parse table from html" -> https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/ e copiato il codice XD

import requests
import pandas as pd
from bs4 import BeautifulSoup

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [37]:
hp = HTMLTableParser()

parsed_table = hp.parse_html_table(tables[5])

In [38]:
parsed_table[:50]

,Rank[Note 2]\n,Mountain\n,Height(m) [2]\n,Drop (m)[Note 3]\n,Coordinates[4]\n,Range [Note 4]\n,Range[Note 5]\n,Region [Note 6]\n,Country\n,First[5]ascent\n
0,1\n,Mont Blanc / Monte Bianco\n,4808.0,4695.0,45°49′57″N 06°51′52″E﻿ / ﻿45.83250°N 6.86444°E...,Mont Blanc massif\n,I/B-07.V-B\n,Haute-Savoie/Aosta Valley\n,FR/IT\n,1786\n
1,2\n,Monte Rosa / Mont Rose (Dufourspitze)\n,4634.0,2165.0,45°56′13″N 07°52′01″E﻿ / ﻿45.93694°N 7.86694°E...,Monte Rosa Alps\n,I/B-09.III-A\n,Valais/Piedmont\n,CH\n,1855\n
2,3\n,Dom\n,4545.0,1046.0,46°05′38″N 07°51′32″E﻿ / ﻿46.09389°N 7.85889°E...,Mischabel\n,I/B-09.V-A\n,Valais\n,CH\n,1858\n
3,4\n,Lyskamm\n,4533.0,381.0,45°55′20″N 07°50′08″E﻿ / ﻿45.92222°N 7.83556°E...,Monte Rosa Alps\n,I/B-09.III-A\n,Valais/Aosta Valley\n,CH/IT\n,1861\n
4,5\n,Weisshorn\n,4506.0,1235.0,46°06′05″N 07°42′57″E﻿ / ﻿46.10139°N 7.71583°E...,Weisshorn-Matterhorn\n,I/B-09.II-D\n,Valais\n,CH\n,1861\n
5,6\n,Matterhorn / Cervino / Cervin\n,4478.0,1042.0,45°58′35″N 07°39′31″E﻿ / ﻿45.97639°N 7.65861°E...,Weisshorn-Matterhorn\n,I/B-09.II-A\n,Valais/Aosta Valley\n,CH/IT\n,1865\n
6,7\n,Dent Blanche\n,4357.0,915.0,46°02′03″N 07°36′43″E﻿ / ﻿46.03417°N 7.61194°E...,Weisshorn-Matterhorn\n,I/B-09.II-C\n,Valais\n,CH\n,1862\n
7,8\n,Grand Combin\n,4314.0,1517.0,45°56′15″N 07°17′57″E﻿ / ﻿45.93750°N 7.29917°E...,Grand Combin Alps\n,I/B-09.I-B\n,Valais\n,CH\n,1859\n
8,9\n,Finsteraarhorn\n,4274.0,2280.0,46°32′14″N 08°07′34″E﻿ / ﻿46.53722°N 8.12611°E...,Bernese Alps\n,I/B-12.II-A\n,Bern/Valais\n,CH\n,1829\n
9,10\n,Zinalrothorn\n,4221.0,490.0,46°03′54″N 07°41′25″E﻿ / ﻿46.06500°N 7.69028°E...,Weisshorn-Matterhorn\n,I/B-09.II-D\n,Valais\n,CH\n,1864\n


In [24]:
import json
#googalto pandas write to file as csv

parsed_table.to_csv('mountains.csv')